In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cpu')

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.416868
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.234219
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.729493
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.594818
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.622055
[1] Test Loss: 0.2112, Accuracy: 93.91%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.435744
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.458099
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.514177
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.202484
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.636772
[2] Test Loss: 0.1294, Accuracy: 96.21%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.354347
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.345969
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.298851
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.196168
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.403422
[3] Test Loss: 0.1024, Accuracy: 96.78%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.413557
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.243840
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.259111


Train Epoch: 29 [38400/60000 (64%)]	Loss: 0.110547
Train Epoch: 29 [51200/60000 (85%)]	Loss: 0.243812
[29] Test Loss: 0.0368, Accuracy: 98.89%
Train Epoch: 30 [0/60000 (0%)]	Loss: 0.013624
Train Epoch: 30 [12800/60000 (21%)]	Loss: 0.091851
Train Epoch: 30 [25600/60000 (43%)]	Loss: 0.215468
Train Epoch: 30 [38400/60000 (64%)]	Loss: 0.139024
Train Epoch: 30 [51200/60000 (85%)]	Loss: 0.073828
[30] Test Loss: 0.0373, Accuracy: 98.88%
Train Epoch: 31 [0/60000 (0%)]	Loss: 0.145705
Train Epoch: 31 [12800/60000 (21%)]	Loss: 0.084540
Train Epoch: 31 [25600/60000 (43%)]	Loss: 0.102447
Train Epoch: 31 [38400/60000 (64%)]	Loss: 0.059657
Train Epoch: 31 [51200/60000 (85%)]	Loss: 0.169568
[31] Test Loss: 0.0381, Accuracy: 98.82%
Train Epoch: 32 [0/60000 (0%)]	Loss: 0.103886
Train Epoch: 32 [12800/60000 (21%)]	Loss: 0.241569
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.136069
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.051764
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.059506
[32] Test Loss: 0.0360, A